In [2]:
import pandas as pd
import mysql.connector
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import locale

# Establish database connection
db_connection = mysql.connector.connect(
    host="localhost",  
    user="root",       
    password="",       
    database="jurujual_pos"  
)

# SQL query to fetch data
query = """
SELECT s.date AS Tanggal, s.reference AS Reference, s.customer_name AS Pembeli, 
    p.product_unit AS Satuan, p.product_name AS Produk, c.category_name AS Kategori,
    sd.quantity AS Qty, p.product_price AS HargaJual, sd.product_discount_amount AS Diskon,
    sp.amount AS SubTotal, s.payment_method AS MetodePembayaran, 
    s.status AS Status, s.payment_status AS StatusPembayaran
    
FROM sales s
JOIN sale_details sd ON s.id = sd.sale_id
JOIN sale_payments sp ON s.id = sp.sale_id
JOIN products p ON sd.product_id = p.id
JOIN categories c ON p.category_id = c.id;
"""

# Reading data into a DataFrame
df = pd.read_sql(query, con=db_connection)
db_connection.close()

# Preprocessing the date column and adding month and year columns
df['Tanggal'] = pd.to_datetime(df['Tanggal'])
df['Bulan'] = df['Tanggal'].dt.month
df['Tahun'] = df['Tanggal'].dt.year

# Menghitung penjualan bulanan untuk setiap produk
monthly_sales = df.groupby(['Tahun', 'Bulan', 'Produk']).agg({
    'SubTotal': 'sum', 
    'Reference': 'nunique',
    'Pembeli': 'nunique',
    'Satuan': 'nunique',
    'Kategori': 'nunique',
    'MetodePembayaran': 'nunique',
    'Qty': 'sum',
    'HargaJual': 'mean',
    'Diskon': 'mean'
}).reset_index()

# Encoding categorical features
le = LabelEncoder()
for col in monthly_sales.columns:
    if monthly_sales[col].dtype == 'object':
        monthly_sales[col] = le.fit_transform(monthly_sales[col])

# Defining features (X) and target (y)
X = monthly_sales[['Reference','Pembeli','Produk', 'Kategori', 'SubTotal', 'HargaJual', 'Bulan']]
y = monthly_sales['Qty']

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Function to evaluate and return model performance as DataFrame
def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    train_mse = mean_squared_error(y_train, y_train_pred)
    train_rmse = round(train_mse ** 0.5, 2)
    train_r2 = round(r2_score(y_train, y_train_pred), 2)
    train_mae = round(mean_absolute_error(y_train, y_train_pred), 2)
    train_mape = round(np.mean(np.abs((y_train - y_train_pred) / y_train)) * 100, 2)
    
    test_mse = mean_squared_error(y_test, y_test_pred)
    test_rmse = round(test_mse ** 0.5, 2)
    test_r2 = round(r2_score(y_test, y_test_pred), 2)
    test_mae = round(mean_absolute_error(y_test, y_test_pred), 2)
    test_mape = round(np.mean(np.abs((y_test - y_test_pred) / y_test)) * 100, 2)
    
    model_name = model.__class__.__name__
    performance_dict_train = {
        'Model': model_name,
        'Training MSE': train_mse,
        'Training RMSE': train_rmse,
        'Training R-squared': train_r2,
        'Training MAE': train_mae,
        'Training MAPE (%)': train_mape
    }
    
    performance_dict_test = {
        'Model': model_name,
        'Test MSE': test_mse,
        'Test RMSE': test_rmse,
        'Test R-squared': test_r2,
        'Test MAE': test_mae,
        'Test MAPE (%)': test_mape
    }
    
    return pd.DataFrame(performance_dict_train, index=[0]), pd.DataFrame(performance_dict_test, index=[0])

# Initialize empty DataFrames for training and testing results
results_df_train = pd.DataFrame()
results_df_test = pd.DataFrame()

# Models to evaluate
models = [
    LinearRegression(),
    Ridge(alpha=1.0),
    Lasso(alpha=0.1),
    ElasticNet(alpha=0.1, l1_ratio=0.5),
    DecisionTreeRegressor(random_state=42),
    RandomForestRegressor(n_estimators=100, random_state=42),
    GradientBoostingRegressor(n_estimators=100, random_state=42),
    SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1),
    KNeighborsRegressor(n_neighbors=5),
    MLPRegressor(hidden_layer_sizes=(100,100), max_iter=500, random_state=42)
]

# Evaluate each model and append results to respective DataFrames
for model in models:
    model_results_train, model_results_test = evaluate_model(model, X_train, y_train, X_test, y_test)
    results_df_train = pd.concat([results_df_train, model_results_train], axis=0)
    results_df_test = pd.concat([results_df_test, model_results_test], axis=0)

# Reset index of results DataFrames
results_df_train.reset_index(drop=True, inplace=True)
results_df_test.reset_index(drop=True, inplace=True)

# Display the results DataFrames with 2 decimal places
pd.options.display.float_format = '{:.2f}'.format

print("Training Set Evaluation Results:")
print(results_df_train.to_string(index=False))

print("\nTesting Set Evaluation Results:")
print(results_df_test.to_string(index=False))


c:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.69491e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Training Set Evaluation Results:
                    Model   Training MSE  Training RMSE  Training R-squared  Training MAE  Training MAPE (%)
         LinearRegression         339.37          18.42                0.31          5.25              88.69
                    Ridge         339.37          18.42                0.31          5.25              88.69
                    Lasso         339.38          18.42                0.31          5.24              88.10
               ElasticNet         339.38          18.42                0.31          5.24              88.17
    DecisionTreeRegressor           0.00           0.00                1.00          0.00               0.00
    RandomForestRegressor           8.46           2.91                0.98          0.39               5.76
GradientBoostingRegressor          11.87           3.45                0.98          1.24              29.39
                      SVR         168.71          12.99                0.66          0.74      